# NeuralCMS Jupyter Notebook
In this notebook, we will load the pre-trained NeuralCMS model and use it to compute a single interior model of Jupiter. Additionally, we will use it to perform a grid search of interior models consistent with Juno gravity data. NeuralCMS receives seven inputs parameters setting the interior model and outputs the gravity moments and mass of the modeled Jupiter.

Further information can be found in Ziv et. al. 2024.

Contact: <maayan.ziv@weizmann.ac.il>

# Load the trained models
Load the trained model that predicts the gravity moments ($J_2\times10^6$, $-J_4\times10^6$, $J_6\times10^6$, and $-J_8\times10^6$) and the mass$\times10^{-27}$ kg

In [ ]:
import numpy as np
import torch
from models_architecture import SharedNN # import the sharing-based model architecture

net_Js = SharedNN(input_size=7)
net_Js.load_state_dict(torch.load('trained_model_Js.pt',map_location=torch.device('cpu')))
net_Js.eval()

Load the trained model that predicts only the mass (more accurately then the model above)

In [ ]:
from models_architecture import FCNN # import the fully-connected model architecture

net_mass = FCNN(input_size=7,output_size=1)
net_mass.load_state_dict(torch.load('trained_model_mass_only.pt',map_location=torch.device('cpu')))
net_mass.eval()

Helping functions to normalize and un-normalize the inputs:

In [ ]:
# This function receives an array of size N*7, and normalizes the inputs to be between 0-1 using Min-Max, with the bounds of the training dataset
def norm_inputs(input_):
    input_[:,0] = (input_[:,0] - 0.1) / (0.6 - 0.1)
    input_[:,1] = (input_[:,1] - 0.5) / (5 - 0.5)
    input_[:,2] = (input_[:,2] - 0) / (0.12 - 0)
    input_[:,3] = (input_[:,3] - 159) / (190 - 159)
    input_[:,4] = (input_[:,4] - 0.27) / (0.286 - 0.27)
    input_[:,5] = (input_[:,5] - 0.005) / (0.06 - 0.005)
    input_[:,6] = (input_[:,6] - 0.01) / (0.6 - 0.01)
    return input_

# This function receives an array of size N*7, and un-normalizes the inputs to their true values by inverting the Min-Max normalization, using the bounds of the training dataset
def unnorm_inputs(input_):
    input_[:,0] = input_[:,0]*(0.6 - 0.1) + 0.1
    input_[:,1] = input_[:,1]*(5 - 0.05) + 0.5
    input_[:,2] = input_[:,2]*(0.12 - 0) + 0
    input_[:,3] = input_[:,3]*(190 - 159) + 159
    input_[:,4] = input_[:,4]*(0.286 - 0.270) + 0.27
    input_[:,5] = input_[:,5]*(0.06 - 0.005) + 0.005
    input_[:,6] = input_[:,6]*(0.6 - 0.01) + 0.01
    return input_

# Computing a single interior model
![Kernel & front-end diagram](plots/NeuralCMS_inputs_table.png)

As an example, we provided parameter values for a specific model.
Users should replace the parameter values to their interests within the bounds shown above.

In [ ]:
# Set the interior model parameters: make sure that Z1<=Z_dilute!!
Y_proto = 0.27705264
T_1bar = 180.47369
Z1 = 0.014210527
P12 = 0.8
m_dilute = 0.38210526
Z_dilute = 0.15263158
r_core = 0.06947095

# Prepare the input vector and normalize it
input_vec = np.array([[m_dilute, P12, r_core, T_1bar, Y_proto, Z1, Z_dilute]])
input_vec = norm_inputs(input_vec)
input_vec = torch.tensor(input_vec, dtype=torch.float32)

# Make a prediction using NeuralCMS
with torch.no_grad():
    pred = net_Js(input_vec) # Predict the Js and the less accurate mass
    pred[:,4] = torch.reshape(net_mass(input_vec),(-1,)) # Accurately predict the mass using a separate network

# Prepare the outputs
J2 = +pred[0][0].numpy() # J2*10^6
J4 = -pred[0][1].numpy() # J4*10^6
J6 = +pred[0][2].numpy() # J6*10^6
J8 = -pred[0][3].numpy() # J8*10^6
mass = +pred[0][4].numpy() # mass*10^-27 kg

Print NeuralCMS results and compare them with the Juno gravity data (Durante et al., 2020, DOI:[10.1029/2019GL086572](https://ui.adsabs.harvard.edu/link_gateway/2020GeoRL..4786572D/doi:10.1029/2019GL086572)).

In [ ]:
Js_NeuralCMS = [J2, J4, J6, J8, mass] # NeuralCMS predictions
Js_Juno = [14696.5735, -586.6085, 34.2007, -2.422, 1.8982532] # Juno measurements (Durante et al., 2020)

# Define headers
header1 = "Juno"
header2 = "NeuralCMS"

# Print headers
print(f"{header1:<10} {header2:<10}")

# Print values
for value1, value2 in zip(Js_Juno[:5], Js_NeuralCMS[:5]):
    print(f"{value1:<10} {value2:<10}")

# Performing a grid search for interior models consistent with Juno gravity data
We perform a grid search exploring all possible combinations of an equally spaced grid for each of the seven parameters with $m$ grid points.

Helping functions to effectively compute batches of $10^6$ interior model simultaneously

In [ ]:
import itertools
from tqdm import tqdm

# This function generates a single interior model from the 7D gridded data
def generate_grid_data(grids, batch_size):
    for grid_point in itertools.product(*grids):
        yield grid_point

# This function generates a batch of interior models from the 7D gridded data
def batch_generator(generator, batch_size):
    batch = []
    for item in generator:
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

Define the grid for each parameter by the number of grid points and its bounds

In [ ]:
m = 10 # number of grid points for each parameter

# For each parameter we define a list with the bounds and number of grid points:
# [minimum value, maximum value, number of grid points]
Y_proto_bounds = [0.272, 0.284, m]
T_1bar_bounds = [159, 187, m]
# As an example, to restrict a parameter to a specific value, set its number of grid points to 1 and the bounds to the specific value: Z1_bounds = [0.02, 0.02, 1]
Z1_bounds = [0.005, 0.06, m]
P12_bounds = [0.8, 5, m]
m_dilute_bounds = [0.11, 0.6, m]
Z_dilute_bounds = [0.06, 0.45, m] # make sure that Z1<=Z_dilute!!
r_core_bounds = [0, 0.12, m]

# Generate the 7D grid data
mdil = np.linspace(m_dilute_bounds[0], m_dilute_bounds[1], m_dilute_bounds[2])
p12 = np.linspace(P12_bounds[0], P12_bounds[1], P12_bounds[2])
rc = np.linspace(r_core_bounds[0], r_core_bounds[1], r_core_bounds[2])
t1bar = np.linspace(T_1bar_bounds[0], T_1bar_bounds[1], T_1bar_bounds[2])
yproto = np.linspace(Y_proto_bounds[0], Y_proto_bounds[1], Y_proto_bounds[2])
z1 = np.linspace(Z1_bounds[0], Z1_bounds[1], Z1_bounds[2])
zdil = np.linspace(Z_dilute_bounds[0], Z_dilute_bounds[1], Z_dilute_bounds[2])

# Grids for each dimension
dimension_grids = [mdil,p12,rc,t1bar,yproto,z1,zdil]

# 1 million grid points per batch
batch_size = 1000000

# The data generator
grid_data_generator = generate_grid_data(dimension_grids, batch_size)

Run the grid search procedure, and accept models that are within a chosen prediction error from the Juno measurements. We provide the errors on the validation dataset used during training. You can choose either the maximal absolute prediction errors, the $3\sigma$ errors, or the $1\sigma$ errors.
Also, a range to allow deviations from the Juno measurements can be added to the prediction errors.

In [ ]:
Js_Juno = [14696.5735, 586.6085, 34.2007, 2.422, 1.8982532] # Juno measurements (Durante et al., 2020)

# As an example, we will find models that are consistent only with J2*10^6, J4*10^6, and the mass*10^-27 kg
range_from_Juno = [2,1,0,0,0.0005] # Only the non-zero entries are important in this case

# Choose you preferred prediction errors (pred_error_choice):
# set it to 1 for 1-sigma, 3 for 3-sigma, or the default, which is the maximum absolute errors
pred_error_choice = 0
if pred_error_choice == 1:
    pred_errs = [0.78904, 0.049011, 0.0044912, 0.00032655, 7.3513e-05]
elif pred_error_choice == 3:
    pred_errs = 3 * [0.78904, 0.049011, 0.0044912, 0.00032655, 7.3513e-05]
else:
    pred_errs = [7.855,0.53317,0.096413,0.0044255,0.0019912]

# Iterate over grid data in batches
for batch_idx, batch in enumerate(tqdm(batch_generator(grid_data_generator, batch_size), desc="Processing batches"),start=1):

    # Normalize the inputs
    inputs_grid_ = norm_inputs(np.array(batch))
    inputs_grid_ = torch.tensor(inputs_grid_, dtype=torch.float32)

    # Make the prediction for the whole batch
    with torch.no_grad():
        predictions = net_Js(inputs_grid_)
        predictions[:,4] = torch.reshape(net_mass(inputs_grid_),(-1,))

    # Save the predictions and the inputs that are consistent with Juno in temporarily variables
    tmp_pred = predictions[np.where((abs(predictions[:,0]-Js_Juno[0]) <= pred_errs[0]+range_from_Juno[0]) & \
                                    (abs(predictions[:,1]-Js_Juno[1]) <= pred_errs[1]+range_from_Juno[1]) & \
                                    (abs(predictions[:,4]-Js_Juno[4]) <= pred_errs[4]+range_from_Juno[4]))]
    tmp_inputs = inputs_grid_[np.where((abs(predictions[:,0]-Js_Juno[0]) <= pred_errs[0]+range_from_Juno[0]) & \
                                    (abs(predictions[:,1]-Js_Juno[1]) <= pred_errs[1]+range_from_Juno[1]) & \
                                    (abs(predictions[:,4]-Js_Juno[4]) <= pred_errs[4]+range_from_Juno[4]))]

    # If it is the first batch, make the temporarily variables the accumulated lists
    if batch_idx == 1:
        pred = tmp_pred
        inputs_out = unnorm_inputs(tmp_inputs) # Un-normalize the inputs
    # Concatenate the acceptable models from the current batch with the previous models accepted previously
    else:
        pred = torch.cat((pred, tmp_pred), dim=0)
        inputs_out = torch.cat((inputs_out, unnorm_inputs(tmp_inputs)), dim=0) # save the un-normalize the inputs

As a final step, transform the results to numpy arrays.
Now you have two arrays:
- accepted_inputs, sized N*7, where the dimensions are in the following order:
  $m_{\rm dilute}$, $P_{12}$, $r_{\rm core}$, $T_{\rm 1 bar}$, $Y_{\rm proto}$, $Z_1$, and $Z_{\rm dilute}$
- accepted_predictions, sized N*5, where the dimensions are in the following order:
  $J_{2}\times10^{6}$, $-J_{4}\times10^{6}$, $J_{6}\times10^{6}$, $-J_{8}\times10^{6}$, and mass$\times10^{-27}$ kg.

In [ ]:
accepted_inputs = inputs_out.numpy()
accepted_predictions = pred.numpy()